In [9]:
# Hier berechne ich einen Distanzmatrix aus den Datensätzen

In [10]:
import pandas as pd
import data_cleanup as dc
import data_exploration as de

In [11]:
fpath = '../DMS_data/P53_HUMAN_Giacomelli_NULL_Etoposide_2018.csv'
df = pd.read_csv(fpath)

In [12]:
# Auftrennen der Mutante in alte und neue AS
mutations_df = dc.aufteilung_mut_pos(df)

In [18]:
# Gruppieren der Substitutionen, geordnet erst nach alter und dann nach neuer AS mit Inspiration von ChatGPT (.groupby in der Pandas Dokumentation beschreibt nur das groupen nach einem Parameter)
subs_df = mutations_df.groupby(["AS_old", "AS_new"])

# Berechnen des means für jede Kombination
mean_scores = subs_df.DMS_score.mean()
# Es gibt 380 Möglichkeiten, 20 AS mit Reihenfolge und ohne zurücklegen zu kombinieren. Aus den Mittelwerten dieser 380 möglichen Substitutionen soll nun eine Distanzmatrix erstellt werden.

In [19]:
# Da "mean_scores" eine Serie ist, muss diese erst wieder in ein Data Frame umgewandelt werden. Dafür müssen den einzelnen Zeilen wieder konkrete Indizes zugewiesen werden, damit wieder ein Data Frame entsteht.
mean_scores_df = mean_scores.reset_index()

# Pivot erstellt aus unserer in einem Data Frame aufgelisteten Substitutionen ein Data Frame, das in Form einer Tabelle eine Übersicht der Mittelwerte zeigt. Die Reihen sind die alten AS, die Spalten die neuen AS.
mean_substitutions = mean_scores_df.pivot(index="AS_old", columns="AS_new", values= "DMS_score")
dc.rmv_na(mean_substitutions)

print(mean_substitutions.columns)

Index(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q',
       'R', 'S', 'T', 'V', 'W', 'Y'],
      dtype='object', name='AS_new')


b### Ergänzungen von Frido für Distanzmatrix

In [20]:
mean_substitutions.head(20)

de.dms_distance_matrix(mean_substitutions)

ValueError: Die Funktion 'aufteilung_mut_pos()' muss vorher ausgeführt werden.

In [16]:
from sklearn.metrics.pairwise import euclidean_distances

# Berechnen der Distanzen der AA zueinander
dms_distances = euclidean_distances(mean_substitutions.values)

# Umwandeln zurück in einen DF zur besseren Übersicht
mean_substitutions_df = pd.DataFrame(dms_distances, index=mean_substitutions.index, columns=mean_substitutions.index)

# Print the first 20 rows of mean_substitutions
mean_substitutions_df.head(20)


AS_old,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
AS_old,,,,,,,,,,,,,,,,,,,,
A,0.000000,5.118419,1.001127,0.950791,3.865506,1.093999,1.727396,5.869602,1.354001,1.858657,1.982457,1.449749,0.619728,1.422936,2.382516,0.879574,1.428805,3.844111,2.678816,5.224924
C,5.118419,0.000000,5.157128,5.225621,2.266408,4.671423,4.215585,2.799431,5.807398,4.018940,3.878780,4.844397,5.130724,6.064423,3.461968,5.276711,4.375534,2.457430,3.015105,2.006232
D,1.001127,5.157128,0.000000,0.981280,3.758420,1.082337,1.656670,6.041829,1.354857,2.016012,1.948455,1.268876,0.886359,1.481424,2.328884,0.981295,1.516787,3.841171,2.628414,5.168214
E,0.950791,5.225621,0.981280,0.000000,3.851885,0.958826,1.639293,5.929282,1.079586,1.820398,1.895125,1.302363,0.923046,1.366785,2.307710,1.034484,1.386531,3.846769,2.680332,5.261142
F,3.865506,2.266408,3.758420,3.851885,0.000000,3.462147,2.805199,3.117083,4.394297,2.539148,2.594098,3.492566,3.820806,4.605781,2.295875,3.973795,3.162375,1.985099,2.091920,2.351548
G,1.093999,4.671423,1.082337,0.958826,3.462147,0.000000,1.322393,5.478072,1.584928,1.630219,1.579012,1.216704,0.958718,1.847809,1.846426,1.157093,1.073299,3.380896,2.270993,4.789685
H,1.727396,4.215585,1.656670,1.639293,2.805199,1.322393,0.000000,5.192499,2.217342,1.468733,1.719725,1.441529,1.681892,2.346398,1.849399,1.679055,1.373227,3.283270,1.932457,4.188192
I,5.869602,2.799431,6.041829,5.929282,3.117083,5.478072,5.192499,0.000000,6.404813,4.593600,4.560649,5.714493,5.889325,6.628173,4.191128,6.130616,5.075546,2.830290,4.264680,3.487852
K,1.354001,5.807398,1.354857,1.079586,4.394297,1.584928,2.217342,6.404813,0.000000,2.334108,2.444159,1.628534,1.270695,1.173660,2.859190,1.243476,1.835478,4.438679,3.420946,5.984730


In [17]:
mean_substitutions.to_csv('./mean_subst.csv')
mean_substitutions_df.to_csv('./mean_subst_df.csv')

### Kommentar von Enno:
Ich glaube du kannst statt mit mean_distances.T einfach mit der Funktion mean_distances_inverted von Dario arbeiten... Ach und kannst du das ganze noch in ne Funktion umschreiben und in statistical functions importieren? Für das AS Datenset hab ich das schon gemacht. Das brauche ich, damit ich das dann schön und sleak in dem Documentations File einfach als eine Funktion benutzen kann. Das wäre niceeee